In [1]:
cd ..

/home/msi/projects/diplomka


In [2]:
import folium
import glob
import json
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [3]:
files = glob.glob("data/out/geo_data/*.json")

In [4]:
files

['data/out/geo_data/FZK_LCCG2-INFN_T1-2a00:139c:5:585:0:41:2:6-2001:760:4205:254::11.json',
 'data/out/geo_data/CA_SFU_T2-CSCS_LCG2-206.12.127.40-148.187.129.15.json',
 'data/out/geo_data/FZK_LCCG2-INFN_T1-192.108.47.6-131.154.254.12.json',
 'data/out/geo_data/FZK_LCCG2-INFN_T1-192.108.47.6-131.154.254.11.json',
 'data/out/geo_data/FZK_LCCG2-INFN_T1-2a00:139c:5:585:0:41:2:12-2001:760:4205:254::12.json',
 'data/out/geo_data/INFN_T1-CSCS_LCG2-131.154.254.11-148.187.129.15.json',
 'data/out/geo_data/FZK_LCCG2-INFN_T1-2a00:139c:5:585:0:41:2:12-2001:760:4205:254::11.json',
 'data/out/geo_data/CA_SFU_T2-CSCS_LCG2-206.12.127.30-148.187.129.15.json',
 'data/out/geo_data/INFN_T1-CSCS_LCG2-2001:760:4205:254::11-2001:620:808:4129::15.json',
 'data/out/geo_data/FZK_LCCG2-INFN_T1-192.108.47.12-131.154.254.11.json',
 'data/out/geo_data/INFN_T1-CSCS_LCG2-131.154.254.12-148.187.129.15.json',
 'data/out/geo_data/FZK_LCCG2-INFN_T1-2a00:139c:5:585:0:41:2:6-2001:760:4205:254::12.json',
 'data/out/geo_data

In [5]:
all_nodes  = set()

In [6]:
src_dest = set()

In [7]:
multi_site_nodes = defaultdict(list)

In [8]:
node_pos = {}

In [9]:
edges = defaultdict(list)

In [10]:
for file in files:
    with open(file, "r") as f:
        data = json.load(f)
    print(file)
    dataset = tuple(file.split('/')[-1].split('-')[:2])
    src_dest.add(data['src'])
    src_dest.add(data['dest'])
    for node, ip in data['node_mapping'].items():
        if ip in all_nodes:
            multi_site_nodes[ip].append(data['node_pos'][node])
        else:
            all_nodes.add(ip)
            node_pos[ip] = data['node_pos'][node]
    
    edges[dataset] += data['edges']

data/out/geo_data/FZK_LCCG2-INFN_T1-2a00:139c:5:585:0:41:2:6-2001:760:4205:254::11.json
data/out/geo_data/CA_SFU_T2-CSCS_LCG2-206.12.127.40-148.187.129.15.json
data/out/geo_data/FZK_LCCG2-INFN_T1-192.108.47.6-131.154.254.12.json
data/out/geo_data/FZK_LCCG2-INFN_T1-192.108.47.6-131.154.254.11.json
data/out/geo_data/FZK_LCCG2-INFN_T1-2a00:139c:5:585:0:41:2:12-2001:760:4205:254::12.json
data/out/geo_data/INFN_T1-CSCS_LCG2-131.154.254.11-148.187.129.15.json
data/out/geo_data/FZK_LCCG2-INFN_T1-2a00:139c:5:585:0:41:2:12-2001:760:4205:254::11.json
data/out/geo_data/CA_SFU_T2-CSCS_LCG2-206.12.127.30-148.187.129.15.json
data/out/geo_data/INFN_T1-CSCS_LCG2-2001:760:4205:254::11-2001:620:808:4129::15.json
data/out/geo_data/FZK_LCCG2-INFN_T1-192.108.47.12-131.154.254.11.json
data/out/geo_data/INFN_T1-CSCS_LCG2-131.154.254.12-148.187.129.15.json
data/out/geo_data/FZK_LCCG2-INFN_T1-2a00:139c:5:585:0:41:2:6-2001:760:4205:254::12.json
data/out/geo_data/FZK_LCCG2-INFN_T1-192.108.47.12-131.154.254.12.js

In [11]:
for node in multi_site_nodes.keys():
    node_pos[node] = np.array(multi_site_nodes[node]).mean(axis=0).tolist()

In [12]:
import matplotlib

In [13]:
colors = matplotlib.colors.TABLEAU_COLORS

In [14]:
final_nodes = set(all_nodes)
final_nodes ^= set(multi_site_nodes.keys())
final_nodes ^= src_dest

In [15]:
multi_site_nodes_set = set(multi_site_nodes.keys()) ^ src_dest

In [46]:
m = folium.Map(location=[46.02444, 8.96008], height='90%',  zoom_start=3, tiles="Stamen Toner")
folium.TileLayer('OpenStreetMap').add_to(m)
cmap = iter(colors.values())

layer = folium.FeatureGroup(name=f"single trace")
for node in final_nodes:
    folium.Marker(
        node_pos[node], popup=node, tooltip=node, icon=folium.Icon(color='green', icon='t', prefix='fa')).add_to(layer)

layer.add_to(m)
layer = folium.FeatureGroup(name=f"multiple nodes")

for node in multi_site_nodes_set:
    folium.Marker(
            node_pos[node], popup=node, tooltip=node, icon=folium.Icon(color='blue', icon='thumbtack', prefix='fa') 
        ).add_to(layer)

layer.add_to(m)
layer = folium.FeatureGroup(name=f"src/dest nodes")
for node in src_dest:
    folium.Marker(
            node_pos[node], popup=node, tooltip=node, icon=folium.Icon(color='red', icon='flag-checkered', prefix='fa') 
        ).add_to(layer)
layer.add_to(m)

        
bearings = [0, 45, 90, 135, 180, 225, 270, 315]
for (u, v), data in edges.items():
    layer = folium.FeatureGroup(name=f"{u}->{v}")
    color = next(cmap)
    for edge in data:
        pos = [node_pos[edge['src']], node_pos[edge['dest']]]
        e = folium.PolyLine(pos, color=color,  weight=5*edge['prob'], tooltip=edge['prob'], opacity=0.4,     bearings=bearings
        ).add_to(layer) 
        # folium.RegularPolygonMarker(location=node_pos[edge['dest']], color=color, number_of_sides=3, radius=10, ).add_to(m)
    layer.add_to(m)

folium.LayerControl().add_to(m)
# for ip, node_id in ta.hops_ip_model.nodes.items():
    
#     row = meta_df.loc[(meta_df['ip']==ip)].iloc[0]
#     folium.Marker(
#     pos['node_pos'][str(node_id)], popup=row['isp'], tooltip=row['city']
# ).add_to(m)
# for u,v in  ta.hops_ip_model.edges:
#     if u==v:
#         continue
#     row_u = meta_df.loc[(meta_df['ip']==u)].iloc[0]
#     row_v = meta_df.loc[(meta_df['ip']==v)].iloc[0]
#     uid =  ta.hops_ip_model.node_to_index[u]
#     vid =  ta.hops_ip_model.node_to_index[v]


#     tmp = ta.hops_ip_model
#     p = tmp.counts[u][v] / tmp.node_out_counts[u]

#     folium.PolyLine(
#     [ pos['node_pos'][str(uid)], pos['node_pos'][str(vid)]], color='yellow',  weight=5*p, tooltip=p, opacity=0.4


#     ).add_to(m)


In [47]:
m